# How to calculate Risk
We also created a function used to estimate the risk of a certain strategic voting an agent might consider to use instead of its honest preferences.

***

## Preparations

In [1]:
import numpy as np

***

## Example

In [3]:
honest_vote = np.array(["A","B","C","D","E"])
estimated_opponent_votes = np.array([
    ["B","A","C","D","E"],
    ["C","B","A","D","E"],
    ["D","B","C","A","E"],
    ["E","B","C","D","A"]
])

strategic_vote = np.array(["B","A","C","D","E"])

***

In [20]:
def get_result (votings):
    candidates = np.zeros(len(votings[0]))
    i = 0
    for candidate in votings[0]:
        candidates[i] = candidate
        i += 1
    countings = np.zeros(len(votings[0]))
    for vote in votings:
        for rank in vote:
            index = 0
            for candidate in candidates:
                if(rank == candidate):
                    countings[index] += 1
                    break
                index += 1
    result = list(zip(candidates,countings))
    result.sort(key=lambda x:x[1])
    return [x for (x,y) in result]

In [21]:
all_votes = np.concatenate((estimated_opponent_votes,honest_vote.reshape((1,5))),axis=0)
print(get_result(all_votes))

ValueError: could not convert string to float: 'B'

In [ ]:
def get_risk (strategic_vote : np.array, honest_vote : np.array, estimated_honest_outcome : np.array, estimated_opponent_votes : np.array):
    